In [1]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

RR = RealField(5000)
Q.<x> = LaurentPolynomialRing(QQ)

sp_t_poly = 1/2*x**-1 + 1/2*x

# TiGER v2.1
TiGER256 = {}
TiGER256['n'] = 1024
TiGER256['q'] = 256
TiGER256['p'] = 128
TiGER256['k1'] = 128
TiGER256['k2'] = 4
TiGER256['h_s'] = 196
TiGER256['h_r'] = 196
TiGER256['h_e'] = 32
TiGER256['f'] = 5
TiGER256['thres'] = 256/2

TiGER192 = {}
TiGER192['n'] = 1024
TiGER192['q'] = 256
TiGER192['p'] = 128
TiGER192['k1'] = 64
TiGER192['k2'] = 4
TiGER192['h_s'] = 132
TiGER192['h_r'] = 132
TiGER192['h_e'] = 32
TiGER192['f'] = 5
TiGER192['thres'] = 256/2

TiGER128 = {}
TiGER128['n'] = 512
TiGER128['q'] = 256
TiGER128['p'] = 128
TiGER128['k1'] = 64 
TiGER128['k2'] = 16
TiGER128['h_s'] = 142
TiGER128['h_r'] = 110
TiGER128['h_e'] = 32
TiGER128['f'] = 3
TiGER128['thres'] = 256/2


In [2]:
def plot_poly_dist_(poly):
    plt.plot(poly.exponents(), poly.coefficients())
    plt.show()

def unif_error_poly_(p, q):
    
    err_max = int(q/p/2)
    err_list = [i for i in range(err_max+1)]
    poly = 0
    for i in err_list:
        if i == 0 :
            poly += 1
        elif i == err_max :
            poly += 1/2*x**err_max + 1/2*x**(-err_max)
        else :
            poly += x**i + x**(-i)
    
    return poly/(2*err_max)


def e2_poly_(p_prime, q): # Why this way?
    
    err_nums = q/p_prime
    zero_pos = int(err_nums/2-1)
    poly = 0
    for i in range(err_nums):
        if not i == zero_pos:
            poly += x**(i-zero_pos)
        else :
            poly += 1
    
    return poly/err_nums
    
def dist_mult_(poly1, poly2):
    
    poly = 0
    expo_u = vector(poly1.exponents())
    expo_v = vector(poly2.exponents())
    coef_u = vector(poly1.coefficients())
    coef_v = vector(poly2.coefficients())
    
    for i, rows in enumerate(expo_u.tensor_product(expo_v)):
        for j, elts in enumerate(rows):
            poly += coef_u.tensor_product(coef_v)[i][j]*x**elts
    
    return poly

def DGdict_to_DGpoly_(DGdict):
    DGpoly = 0
    for i in DGdict :
        if i == 0 :
            DGpoly += DGdict[i]*x**i
        else :
            DGpoly += DGdict[i]*x**i + DGdict[i]*x**-i
    DGpoly = DGpoly/DGpoly(1)
    
    return DGpoly

def dist_of_squared_(poly):
    res = 0
    for i, expo in enumerate(poly.exponents()):
        res += poly.coefficients()[i]*x**(expo**2)
    
    return res

def bayes_poly_list_2_(post, pri, n):
    res = []
    
    squared_pri = dist_of_squared_(pri)
    sum_squared_pri = squared_pri**n
    
    for expo_post in post.exponents():
        numerator_poly = (squared_pri**(n-1))*x**(expo_post**2)
        temp_res = 0
        
        for expo_pri in sum_squared_pri.exponents():
            temp_res += (numerator_poly[expo_pri]/sum_squared_pri[expo_pri])*x**expo_pri
            
        temp_res *= post[expo_post]
        res.append(temp_res)

        
    return res

def bayes_poly_list_(post, pri, n):
    res = [0 for i in range(n*(max(post.exponents()))**2+1)]
    
    squared_pri = dist_of_squared_(pri)
    sum_squared_pri = squared_pri**n
    
    for expo_post in post.exponents():
        numerator_poly = (squared_pri**(n-1))*x**(expo_post**2)
        
        for expo_pri in sum_squared_pri.exponents():
            #print(expo_pri)
            res[expo_pri] += post[expo_post]*(numerator_poly[expo_pri]/sum_squared_pri[expo_pri])*x**expo_post 
            

        
    return res

def bayes_poly_dict_(post, pri, n):
    res = {}
    
    squared_pri = dist_of_squared_(pri)
    sum_squared_pri = squared_pri**n
    
    for expo_post in post.exponents():
        numerator_poly = (squared_pri**(n-1))*x**(expo_post**2)
        
        for expo_pri in sum_squared_pri.exponents():
            #print(expo_pri)
            res.setdefault((expo_pri, sum_squared_pri[expo_pri]), 0)
            res[(expo_pri, sum_squared_pri[expo_pri])] += post[expo_post]*(numerator_poly[expo_pri]/sum_squared_pri[expo_pri])*x**expo_post 
            

        
    return res
      
def cumul_binom_(p, n, k):
    res = 0
    for i in range(k+1):
        res += binomial(n, i)*(p**i)*(1-p)**(n-i)
        
    return res

def var_of_poly_(poly):
    if poly == 1:
        var = 0
    else :
        var = 0
        for poly_expo in poly.exponents():
            var += (poly_expo**2)*poly[poly_expo]
        
    return var

In [5]:
def TiGER_DFP(scheme):
    n = scheme['n']
    p = scheme['p']
    q = scheme['q']
    k1 = scheme['k1']
    k2 = scheme['k2']
    h_r = scheme['h_r']
    h_e = scheme['h_e']
    h_s = scheme['h_s']
    f = scheme['f']
    #f = 0
    thres = scheme['thres']
    
    uA_bayes = unif_error_poly_(p, q)
    uA_bayes = bayes_poly_dict_(uA_bayes, uA_bayes, n)
    
    uB_prime_bayes =  unif_error_poly_(k1, q)
    uB_prime_bayes = bayes_poly_dict_(uB_prime_bayes, uB_prime_bayes, n)
    
    uB_double_prime = unif_error_poly_(k2, q)
    
    e1 = (h_e/n)*sp_t_poly+1-h_e/n
    s = (h_s/n)*sp_t_poly+1-h_s/n
    
    e1s = s**h_e
    
    g = uB_double_prime*e1
    
    g = g**2
    #print(g)
    
    pfail = 0
    
      
    for (uA, uA_prob) in tqdm(uA_bayes, leave=False):
        for (uB_prime, uB_prime_prob) in uB_prime_bayes:
            pbitfail = 0
            
            if uA_prob*uB_prime_prob < 2**-300:
                continue
            
            
            #print(uA, uA_bayes[(uA, uA_prob)])
            #print(uB_prime, uB_prime_bayes[(uB_prime, uB_prime_prob)])
            
            #gaussian_std = sqrt(RR(h_r*var_of_poly_(uA_bayes[(uA, uA_prob)])+h_s*var_of_poly_(uB_prime_bayes[(uB_prime, uB_prime_prob)])+h_s*var_of_poly_(e1)+var_of_poly_(g)))
            
            #g 텀 뺀거
            # D2 인코딩이면 오류가 두배?
            gaussian_std = sqrt(RR(2*(h_r*var_of_poly_(uA_bayes[(uA, uA_prob)])+h_s*var_of_poly_(uB_prime_bayes[(uB_prime, uB_prime_prob)])+h_s*var_of_poly_(e1))))
            # 오류 두배 안 한것
            #gaussian_std = sqrt(RR(h_r*var_of_poly_(uA_bayes[(uA, uA_prob)])+h_s*var_of_poly_(uB_prime_bayes[(uB_prime, uB_prime_prob)])+h_s*var_of_poly_(e1)))
            
            T = RealDistribution('gaussian', gaussian_std)
            
            for g_expo in g.exponents():
                pbitfail += RR(1-T.cum_distribution_function(RR(thres-g_expo))+T.cum_distribution_function(RR(-thres-g_expo)))*RR(g[g_expo])
                
            pfail += RR((1-cumul_binom_(pbitfail, n, f)))*RR(uA_prob)*RR(uB_prime_prob)
            #print(log(pfail,2).n(100))
    
    return pfail.n(1000), log(pfail, 2).n(1000)

In [6]:
TiGER_DFP(TiGER128)

  0%|          | 0/513 [00:00<?, ?it/s]

(6.70463846104628546679219269130365554002355760876669110718092316963395718659902090932599813945468571841256212730697672434784593408195529210227145581796108957029162942657742771754840028547200891521188353788984607109244042104231321883430113236826243969660138347389464458377105892195289928291041951739916e-18,
 -57.0495461692086797464113853782274370872327002977524951970639928214347012856511014604327737151302127377200198237695213434393825839817702249961920426916044412287746813694675847205100372460048777147242715105601432520147384083130273208269063589589670725190225442509784269808049516829303059945898849443031)

In [7]:
TiGER_DFP(TiGER192)

  0%|          | 0/1025 [00:00<?, ?it/s]

(1.36979993314068961396173484239839873306200393374437317996769083497358891950345049773315456619932161433037483970792195898370830963913671563750552926197841517704095281525750450943964550015647066386263860733242015111303011739172092518142266249322256927564191262157695536295874284921664589272912655424503e-10,
 -32.7653157538945007200901041239525740071530114792524300357038788403944598264974160850952819228829599961921346743866086757251949432558264212011104140630033875151145068749339753853981270786005175352877554451659525124414899784995588923591968448539711436187883095631930523091116208182970227065929607211825)

In [8]:
TiGER_DFP(TiGER256)

  0%|          | 0/1025 [00:00<?, ?it/s]

(2.35653850082936776694819514608378114849586756141504103359976533163581835405379930584595744332066900326690499725139994661154061601423103312899578389338636431521765768797884175236988057101272721879414972730328282515505156541366081300738057772622575994476491983554082493489077734176607178531849674275547e-15,
 -48.5922521711946604113942223426402187861513954214346817692385910368745022793376048665040611882967637927247764377784443319508029311710900237275094260460142594374943043469977375882040437327704779963310866579126165433280996164986630168908411559958132300565624520727852343791842286434101737487391724668671)